<a href="https://colab.research.google.com/github/wasihun-code/BLOG_Flask/blob/main/EDA%20RP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clearn Stations

In [6]:
import pandas as pd
import re

# Load your stations file
df = pd.read_csv("/content/drive/My Drive/EDA-RP/Dataset/stations.csv")

# Filter only Delhi stations
df_delhi = df[df['city'].str.contains("delhi", case=False, na=False)].copy()

# Clean station names for matching
def clean_name(name):
    name = name.replace("Delhi", "")
    name = re.sub(r"-\s*(DPCC|CPCB|IMD|IITM)", "", name, flags=re.IGNORECASE)
    name = name.replace(",", " ")
    name = re.sub(r"\s+", " ", name).strip()
    return name

df_delhi["clean_station"] = df_delhi["station"].apply(clean_name)

# Keep only essential columns
df_final = df_delhi[["clean_station", "station", "latitude", "longitude"]].drop_duplicates()

# Save in Drive
output_path = "/content/drive/My Drive/EDA-RP/Dataset/stations_delhi_clean.csv"
df_final.to_csv(output_path, index=False)

output_path, df_final.head()


('/content/drive/My Drive/EDA-RP/Dataset/stations_delhi_clean.csv',
    clean_station                    station   latitude  longitude
 69        Alipur       Alipur, Delhi - DPCC  28.815329  77.153010
 70   Anand Vihar  Anand Vihar, Delhi - DPCC  28.647622  77.315809
 72   Ashok Vihar  Ashok Vihar, Delhi - DPCC  28.695381  77.181665
 93     Najafgarh    Najafgarh, Delhi - DPCC  28.570173  76.933762
 94        Narela       Narela, Delhi - DPCC  28.822836  77.101981)

# Assign neraest station to ward

In [8]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from geopy.distance import geodesic
import numpy as np

# -------------------------------------------------------------
# Load ward data and station coordinates
# -------------------------------------------------------------
ward_geo = "/content/drive/My Drive/EDA-RP/Dataset/ward/cleaned/ward_master_geo.geojson"
station_file = "/content/drive/My Drive/EDA-RP/Dataset/stations_delhi_clean.csv"

# ❗ FIXED: Load GeoJSON using gpd.read_file, NOT pandas
wards = gpd.read_file(ward_geo)

stations = pd.read_csv(station_file)

# -------------------------------------------------------------
# Filter stations → only those with PM2.5 hourly data
# -------------------------------------------------------------
available = [
 'alipur',
 'ashok_vihar',
 'bawana',
 'dr._karni_singh_shooting_range',
 'ihbas_dilshad_garden',
 'ito',
 'lodhi_road',
 'mandir_marg',
 'narela',
 'north_campus_du',
 'nsit_dwarka',
 'punjabi_bagh',
 'r_k_puram'
]

stations["clean_station"] = (
    stations["clean_station"]
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
    .str.replace("(", "")
    .str.replace(")", "")
)

stations = stations[stations["clean_station"].isin(available)]

print("Stations used:", stations["clean_station"].tolist())

# -------------------------------------------------------------
# Compute ward centroids
# -------------------------------------------------------------
wards["centroid"] = wards.geometry.centroid
wards["lat"] = wards["centroid"].y
wards["lon"] = wards["centroid"].x

# -------------------------------------------------------------
# Compute nearest AVAILABLE station for each ward
# -------------------------------------------------------------
def nearest_station(lat, lon):
    dists = []
    for _, row in stations.iterrows():
        s_lat = row["latitude"]
        s_lon = row["longitude"]
        dist = geodesic((lat, lon), (s_lat, s_lon)).km
        dists.append((dist, row["clean_station"]))
    return min(dists, key=lambda x: x[0])[1]

wards["nearest_available_station"] = wards.apply(
    lambda r: nearest_station(r["lat"], r["lon"]), axis=1
)

# -------------------------------------------------------------
# Save new ward → AVAILABLE station mapping
# -------------------------------------------------------------
OUT = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_station_mapping_AVAILABLE.csv"
wards[["ward_no", "ward_name", "lat", "lon", "nearest_available_station"]].to_csv(OUT, index=False)

print("Saved:", OUT)
wards.head()


Stations used: ['alipur', 'ashok_vihar', 'narela', 'north_campus_du', 'r_k_puram', 'dr._karni_singh_shooting_range', 'ihbas_dilshad_garden', 'lodhi_road', 'lodhi_road', 'mandir_marg', 'ito', 'nsit_dwarka', 'bawana', 'punjabi_bagh']


/tmp/ipython-input-499683967.py:53: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  wards["centroid"] = wards.geometry.centroid


Saved: /content/drive/My Drive/EDA-RP/Dataset/ward/ward_station_mapping_AVAILABLE.csv


,ward_name,ward_no,area_sqkm,population,pop_density,slum_clusters,geometry,centroid,lat,lon,nearest_available_station
0,DELHI CANTT CHARGE 1,1,1.659836,55512.0,33444.259497,0,"POLYGON ((77.13228 28.63154, 77.13644 28.62062...",POINT (77.1318 28.62141),28.621415,77.131800,north_campus_du
1,DELHI CANTT CHARGE 2,2,11.405083,37929.0,3325.622550,0,"POLYGON ((77.15429 28.62335, 77.15501 28.62228...",POINT (77.14289 28.61059),28.610587,77.142895,north_campus_du
2,DELHI CANTT CHARGE 4,4,10.902895,NaN,NaN,0,"POLYGON ((77.15755 28.57578, 77.15672 28.57564...",POINT (77.14556 28.56317),28.563168,77.145558,r_k_puram
3,DELHI CANTT CHARGE 5,5,4.545305,NaN,NaN,0,"POLYGON ((77.1348 28.57051, 77.13429 28.57048,...",POINT (77.13216 28.57656),28.576558,77.132160,r_k_puram
4,DELHI CANTT CHARGE 6,6,19.836437,NaN,NaN,0,"POLYGON ((77.12157 28.59308, 77.12878 28.59029...",POINT (77.10751 28.57027),28.570268,77.107509,r_k_puram


# Build ward daily pm2.5

In [9]:
import pandas as pd
import glob
import numpy as np

# ---------------------------------------------------------
# 1. Load UPDATED ward → station mapping
# ---------------------------------------------------------
MAP = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_station_mapping_AVAILABLE.csv"
ward_map = pd.read_csv(MAP)

# Normalize station name
ward_map["station_clean"] = (
    ward_map["nearest_available_station"]
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
    .str.replace("(", "")
    .str.replace(")", "")
)

print("Unique stations (updated mapping):", ward_map["station_clean"].unique()[:15])


# ---------------------------------------------------------
# 2. Load hourly PM2.5 files
# ---------------------------------------------------------
folder = "/content/drive/My Drive/EDA-RP/Dataset/hourly/"
files = glob.glob(folder + "*.csv")

print("Found hourly files:", len(files))

dfs = []


for f in files:
    df = pd.read_csv(f)

    # Extract station name from filename
    raw = f.split("_site_")[1].split("_Delhi")[0]
    st = "_".join(raw.split("_")[1:])     # remove station ID

    st_clean = (
        st.lower()
        .replace("-", "_")
        .replace(" ", "_")
        .replace("__", "_")
        .strip()
    )

    df["station_clean"] = st_clean

    # Parse datetime and PM2.5
    df["date"] = pd.to_datetime(df["Timestamp"], errors="coerce")
    df["pm25"] = pd.to_numeric(df["PM2.5 (µg/m³)"], errors="coerce")

    dfs.append(df[["date", "station_clean", "pm25"]])


stations_hourly = pd.concat(dfs, ignore_index=True)
print("Hourly rows:", stations_hourly.shape)



# ---------------------------------------------------------
# 3. Convert hourly → daily mean
# ---------------------------------------------------------
stations_daily = (
    stations_hourly
    .groupby(["station_clean", stations_hourly["date"].dt.date])
    .pm25.mean()
    .reset_index()
    .rename(columns={"date": "day"})
)

print("Daily rows:", stations_daily.shape)
print(stations_daily.head())


# ---------------------------------------------------------
# 4. Merge UPDATED mapping → daily PM2.5
# ---------------------------------------------------------
ward_daily = ward_map.merge(
    stations_daily,
    left_on="station_clean",
    right_on="station_clean",
    how="left"
)


# ---------------------------------------------------------
# 5. Save updated ward_daily_pm25
# ---------------------------------------------------------
OUT1 = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_daily_pm25.csv"
ward_daily.to_csv(OUT1, index=False)

print("Saved updated ward_daily_pm25:", OUT1)


Unique stations (updated mapping): ['north_campus_du' 'r_k_puram' 'nsit_dwarka' 'ito' 'mandir_marg'
 'lodhi_road' 'punjabi_bagh' 'ashok_vihar' 'ihbas_dilshad_garden'
 'dr._karni_singh_shooting_range' 'bawana' 'alipur' 'narela']
Found hourly files: 13
Hourly rows: (113880, 3)
Daily rows: (4745, 3)
  station_clean         day        pm25
0        alipur  2023-01-01  130.500000
1        alipur  2023-01-02  193.333333
2        alipur  2023-01-03  211.177083
3        alipur  2023-01-04  146.750000
4        alipur  2023-01-05  154.880435
Saved updated ward_daily_pm25: /content/drive/My Drive/EDA-RP/Dataset/ward/ward_daily_pm25.csv


## Compute annual PM2.5 per ward

In [10]:
import pandas as pd
import numpy as np

# Load ward_daily_pm25
daily = pd.read_csv("/content/drive/My Drive/EDA-RP/Dataset/ward/ward_daily_pm25.csv")

# Convert day to datetime
daily["day"] = pd.to_datetime(daily["day"], errors="coerce")

# Compute annual mean PM2.5
annual = (
    daily.groupby("ward_no")["pm25"]
    .mean()
    .reset_index()
    .rename(columns={"pm25": "pm25_annual"})
)

# Save
OUT = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_annual_pm25.csv"
annual.to_csv(OUT, index=False)

print("Saved:", OUT)
annual.head()


Saved: /content/drive/My Drive/EDA-RP/Dataset/ward/ward_annual_pm25.csv


,ward_no,pm25_annual
0,1.0,105.887117
1,2.0,100.471560
2,3.0,107.665114
3,4.0,104.019682
4,5.0,108.133324


## Identify stations that actually have PM2.5 files

In [11]:
import glob
import pandas as pd

folder = "/content/drive/My Drive/EDA-RP/Dataset/hourly/"

files = glob.glob(folder + "*.csv")

stations_available = []
for f in files:
    raw = f.split("_site_")[1].split("_Delhi")[0]   # extract station part
    st = "_".join(raw.split("_")[1:])              # drop site ID

    st_clean = (
        st.lower()
        .replace("-", "_")
        .replace(" ", "_")
        .replace("(", "")
        .replace(")", "")
        .replace("__", "_")
    )

    stations_available.append(st_clean)

stations_available = sorted(set(stations_available))

print("Stations with PM2.5 hourly data:\n", stations_available)


Stations with PM2.5 hourly data:
 ['alipur', 'ashok_vihar', 'bawana', 'dr._karni_singh_shooting_range', 'ihbas_dilshad_garden', 'ito', 'lodhi_road', 'mandir_marg', 'narela', 'north_campus_du', 'nsit_dwarka', 'punjabi_bagh', 'r_k_puram']


# Create master dataset for analysis


In [12]:
import pandas as pd

# Load datasets
WARD = "/content/drive/My Drive/EDA-RP/Dataset/ward/cleaned/ward_master_dataset.csv"
PM = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_annual_pm25.csv"

df_ward = pd.read_csv(WARD)
df_pm = pd.read_csv(PM)

# Merge ward + annual pm2.5
df = df_ward.merge(df_pm, on="ward_no", how="left")

# Use annual PM2.5 (correct column)
df["pm25_exposure"] = df["pm25_annual"]

# Fix missing population + slum
df["population"] = df["population"].fillna(0)
df["slum_clusters"] = df["slum_clusters"].fillna(0)

# Population-weighted exposure
df["pop_weighted_pm25"] = df["pm25_exposure"] * df["population"]

# Slum–weighted exposure (optional metric)
df["slum_exposure_index"] = df["pm25_exposure"] * (1 + df["slum_clusters"])

# Save final master
OUT_MASTER = "/content/drive/My Drive/EDA-RP/Dataset/ward/ward_exposure_master.csv"
df.to_csv(OUT_MASTER, index=False)

print("Saved:", OUT_MASTER)
df.head()


Saved: /content/drive/My Drive/EDA-RP/Dataset/ward/ward_exposure_master.csv


,ward_no,ward_name,area_sqkm,population,pop_density,slum_clusters,pm25_annual,pm25_exposure,pop_weighted_pm25,slum_exposure_index
0,1.0,DELHI CANTT CHARGE 1,1.659836,55512.0,33444.259497,0,105.887117,105.887117,5.878006e+06,105.887117
1,2.0,DELHI CANTT CHARGE 2,11.405083,37929.0,3325.622550,0,100.471560,100.471560,3.810786e+06,100.471560
2,4.0,DELHI CANTT CHARGE 4,10.902895,0.0,NaN,0,104.019682,104.019682,0.000000e+00,104.019682
3,5.0,DELHI CANTT CHARGE 5,4.545305,0.0,NaN,0,108.133324,108.133324,0.000000e+00,108.133324
4,6.0,DELHI CANTT CHARGE 6,19.836437,0.0,NaN,0,106.823648,106.823648,0.000000e+00,106.823648


# Compute Exposure Inequality

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/My Drive/EDA-RP/Dataset/ward/ward_exposure_master.csv")

# ---- Gini function (weighted + safe) ----
def gini(x, w=None):
    x = np.asarray(x)
    if w is None:
        w = np.ones_like(x)

    # Prevent zero-weights
    w = np.where(w <= 0, 1, w)

    sorted_idx = np.argsort(x)
    x, w = x[sorted_idx], w[sorted_idx]

    cumw = np.cumsum(w)
    cumxw = np.cumsum(x * w)

    g = 1 - 2 * np.sum(cumxw * w) / (cumxw[-1] * cumw[-1]) + (cumw[-1] + w[0]) / cumw[-1]
    return g

# Replace population zeros (just for weighted Gini)
df["pop_fixed"] = df["population"].replace(0, 1)

# ---- Compute Ginis ----
gini_unweighted = gini(df["pm25_exposure"])
gini_popweighted = gini(df["pm25_exposure"], df["pop_fixed"])

print("Gini (Ward-level, unweighted):", gini_unweighted)
print("Gini (Population-weighted):", gini_popweighted)


In [15]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/drive/My Drive/EDA-RP/Dataset/ward/ward_exposure_master.csv")

# Drop rows with missing exposure
df = df.dropna(subset=["pm25_exposure"])

x = df["pm25_exposure"].values
w = df["population"].values

# -----------------------------
# CORRECT WEIGHTED GINI FORMULA
# -----------------------------
def gini_weighted(x, w=None):
    x = np.asarray(x)
    if w is None:
        w = np.ones_like(x)
    else:
        w = np.asarray(w)

    # Sort x and weights
    sorted_idx = np.argsort(x)
    x = x[sorted_idx]
    w = w[sorted_idx]

    # Cumulative weight
    cumw = np.cumsum(w)
    sumw = cumw[-1]

    # Relative cumulative weight
    cumw_rel = cumw / sumw

    # Relative weighted x
    xw = x * w
    cumxw = np.cumsum(xw)
    sumxw = cumxw[-1]
    cumxw_rel = cumxw / sumxw

    # The area between Lorenz curve and equality line
    B = np.trapz(cumxw_rel, cumw_rel)
    A = 0.5 - B

    g = A / (0.5)  # normalize to 0–1
    return g


gini_unweighted = gini_weighted(x)
gini_population_weighted = gini_weighted(x, w)

print("🔥 Corrected Gini (Unweighted):", gini_unweighted)
print("🔥 Corrected Gini (Weighted):", gini_population_weighted)


🔥 Corrected Gini (Unweighted): 0.07535972635181876
🔥 Corrected Gini (Weighted): 0.07608170554259264


/tmp/ipython-input-58304256.py:41: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  B = np.trapz(cumxw_rel, cumw_rel)
